In [113]:
import folium, pickle, os
import pandas as pd
import numpy as np

center = [37.54815556, 126.851675]
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']

# visualization ---- 2.
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    tiles = tiles[2]
)
geo = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

folium.GeoJson(
    geo,
    name = 'geo',
    style_function = lambda x: {'fillColor':'#00000000',
    							'color':'black',
   							    'weight' :'1'}
).add_to(m)

# 저장한 스케쥴 데이터셋을 불러와서 하나로 합치자.

In [114]:
import pickle 

# Pickle 파일에서 불러오기
with open('../data/결과 딕셔너리/청소일정df.pkl', 'rb') as pickle_file:
    saved_data = pickle.load(pickle_file)
saved_data

{'강남구': {'개포로':          SO2        CO        O3       NO2       PM10       PM25  지역명  year  \
  0   0.002397  0.394227  0.043772  0.014152  23.441079  17.454079    0  2024   
  1   0.002384  0.384642  0.043162  0.014789  22.594750  17.097129    0  2024   
  2   0.002378  0.401957  0.041564  0.015984  25.124180  18.016777    0  2024   
  3   0.002381  0.400369  0.042040  0.015178  23.682902  17.744108    0  2024   
  4   0.002378  0.394962  0.042802  0.015166  22.397569  17.908130    0  2024   
  ..       ...       ...       ...       ...        ...        ...  ...   ...   
  56  0.002300  0.302992  0.029164  0.011431  10.788312  14.415910    0  2024   
  57  0.002322  0.292409  0.028973  0.012098  11.255184  14.417664    0  2024   
  58  0.002348  0.310578  0.027881  0.013349  15.155076  15.717018    0  2024   
  59  0.002381  0.311608  0.028943  0.012623  15.107582  15.832557    0  2024   
  60  0.002406  0.310445  0.030362  0.012710  15.204810  16.380282    0  2024   
  
      month

In [115]:
final_df = pd.DataFrame()

for goo in saved_data.keys():
    for road in saved_data[goo].keys():
        saved_data[goo][road]['지역명'] = goo
        saved_data[goo][road]['도로명'] = road
        saved_data[goo][road]['도로주소'] = saved_data[goo][road]['지역명'] + " " + saved_data[goo][road]['도로명']
        final_df = pd.concat([final_df, saved_data[goo][road]])

final_df

,SO2,CO,O3,NO2,PM10,PM25,지역명,year,month,day,도로명,기온,습도,평균속도,최고속도,weekday,예측 오염범례,청소유무,청소일정,도로주소
0,0.002397,0.394227,0.043772,0.014152,23.441079,17.454079,강남구,2024,6,3,개포로,19.866973,69.564239,20.637480,30.0,0,4.0,True,True,강남구 개포로
1,0.002384,0.384642,0.043162,0.014789,22.594750,17.097129,강남구,2024,6,4,개포로,19.660488,69.639150,20.637480,30.0,1,4.0,True,False,강남구 개포로
2,0.002378,0.401957,0.041564,0.015984,25.124180,18.016777,강남구,2024,6,5,개포로,19.359115,71.540233,20.637480,30.0,2,4.0,True,False,강남구 개포로
3,0.002381,0.400369,0.042040,0.015178,23.682902,17.744108,강남구,2024,6,6,개포로,19.483676,72.558810,20.637480,30.0,3,4.0,True,False,강남구 개포로
4,0.002378,0.394962,0.042802,0.015166,22.397569,17.908130,강남구,2024,6,7,개포로,19.718750,71.635896,20.637480,30.0,4,4.0,True,False,강남구 개포로
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0.001972,0.282032,0.029267,0.011067,10.552296,14.317289,중랑구,2024,7,29,중랑천로,30.022380,82.675758,20.681980,30.0,0,3.0,False,False,중랑구 중랑천로
57,0.001965,0.273037,0.028444,0.011797,11.103571,14.075469,중랑구,2024,7,30,중랑천로,29.967766,82.212765,20.681980,30.0,1,4.0,True,False,중랑구 중랑천로
58,0.001974,0.286303,0.027935,0.012349,13.553137,14.621507,중랑구,2024,7,31,중랑천로,29.877163,83.220053,20.681980,30.0,2,4.0,True,False,중랑구 중랑천로
59,0.002001,0.290148,0.028765,0.011736,14.158122,15.146303,중랑구,2024,8,1,중랑천로,30.188081,83.633117,20.650186,30.0,3,4.0,True,True,중랑구 중랑천로


자 이제 좌표값을 넣어주자

In [116]:
import requests
import json
import pandas as pd

def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    headers = {"Authorization": "KakaoAK 46ab9da9aff4a67ef7b896c0cb0ce412"}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        api_json = response.json()
        
        # 디버깅용 출력
        # print(f"Response for {address}: {api_json}")

        if api_json['documents']:
            address = api_json['documents'][0]['road_address']
            if address:
                crd = {"lat": address['y'], "lng": address['x']}
                # print('crd:', crd)
                return crd
        return {"lat": 0, "lng": 0}
    except (requests.RequestException, KeyError, IndexError, TypeError) as e:
        print(f"Error with address {address}: {e}")
        return {"lat": 0, "lng": 0}

def add_lat_lng_to_df(df, address_column):
    # Create empty columns for latitude and longitude
    df['lat'] = None
    df['lng'] = None

    result_list = []

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        address = row[address_column]
        location = get_location(address)
        print('location:', location)
        df.loc[index, 'lat'] = location['lat']
        df.loc[index, 'lng'] = location['lng']
        
        result_list.append((location['lat'], location['lng']))

    return result_list

In [117]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21899 entries, 0 to 60
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   SO2      21899 non-null  float64
 1   CO       21899 non-null  float64
 2   O3       21899 non-null  float64
 3   NO2      21899 non-null  float64
 4   PM10     21899 non-null  float64
 5   PM25     21899 non-null  float64
 6   지역명      21899 non-null  object 
 7   year     21899 non-null  int64  
 8   month    21899 non-null  int64  
 9   day      21899 non-null  int64  
 10  도로명      21899 non-null  object 
 11  기온       21899 non-null  float64
 12  습도       21899 non-null  float64
 13  평균속도     21899 non-null  float64
 14  최고속도     21899 non-null  float64
 15  weekday  21899 non-null  int64  
 16  예측 오염범례  21899 non-null  float64
 17  청소유무     21899 non-null  bool   
 18  청소일정     21899 non-null  bool   
 19  도로주소     21899 non-null  object 
dtypes: bool(2), float64(11), int64(4), object(3)
memory u

In [118]:
final_df['지역명'] = final_df['지역명'].astype('string')

In [119]:
# 날짜와 자치구를 입력하면 해당 위치를 출력

def getVisDf(month, day, loca):
    # 여기서 아예 지도 초기화, 출력.
    vis_df = final_df[(final_df['month']==month)&(final_df['day']==day)&(final_df['지역명']==loca)&(final_df['청소일정'])]
    return vis_df

visdf = getVisDf(7, 17, '서초구')
visdf

,SO2,CO,O3,NO2,PM10,PM25,지역명,year,month,day,도로명,기온,습도,평균속도,최고속도,weekday,예측 오염범례,청소유무,청소일정,도로주소
44,0.001651,0.389841,0.031851,0.016937,20.787118,14.797139,서초구,2024,7,17,명달로,27.348115,87.831871,22.379339,50.0,2,4.0,True,True,서초구 명달로
44,0.001651,0.389841,0.031851,0.016937,20.787118,14.797139,서초구,2024,7,17,방배로,27.348115,87.831871,20.929750,50.0,2,4.0,True,True,서초구 방배로


In [120]:
visdf['도로명']

44    명달로
44    방배로
Name: 도로명, dtype: object

In [121]:
real_vis_df = add_lat_lng_to_df(visdf, '도로주소')
real_vis_df

location: {'lat': '37.4842662461058', 'lng': '127.003834177799'}
location: {'lat': '37.4866606902992', 'lng': '126.994293607948'}


/var/folders/db/5_6x6sr56xq_mqcyq8sm1hd40000gn/T/ipykernel_7398/3317793975.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lat'] = None
/var/folders/db/5_6x6sr56xq_mqcyq8sm1hd40000gn/T/ipykernel_7398/3317793975.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lng'] = None
/var/folders/db/5_6x6sr56xq_mqcyq8sm1hd40000gn/T/ipykernel_7398/3317793975.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

[('37.4842662461058', '127.003834177799'),
 ('37.4866606902992', '126.994293607948')]

In [122]:
visdf2 = getVisDf(7, 17, '강남구')
real_vis_df2 = add_lat_lng_to_df(visdf2, '도로주소')
real_vis_df2

location: {'lat': '37.5222932813165', 'lng': '127.036226026992'}


/var/folders/db/5_6x6sr56xq_mqcyq8sm1hd40000gn/T/ipykernel_7398/3317793975.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lat'] = None
/var/folders/db/5_6x6sr56xq_mqcyq8sm1hd40000gn/T/ipykernel_7398/3317793975.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lng'] = None


[('37.5222932813165', '127.036226026992')]

In [123]:
count = 0
for idx, value in visdf2.iterrows():
    # print(coord)
    lat = float(real_vis_df2[count][0]); lng = float(real_vis_df2[count][1])
    print(f"lat {lat} lng {lng}")
    Marker(
        location = [lat, lng],
        # popup = value['도로명']
        icon = folium.Icon('red', icon = 'star'),
    ).add_to(m)
    count += 1
    folium.map.Marker(
    [lat, lng],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 11pt">%s</div>' % str(value['도로명']),
        )
    ).add_to(m)

lat 37.5222932813165 lng 127.036226026992


In [124]:
import ast
from folium.features import DivIcon
from folium import Marker

count = 0
for idx, value in visdf.iterrows():
    # print(coord)
    lat = float(real_vis_df[count][0]); lng = float(real_vis_df[count][1])
    print(f"lat {lat} lng {lng}")
    Marker(
        location = [lat, lng],
        # popup = value['도로명']
        icon = folium.Icon('red', icon = 'star'),
    ).add_to(m)
    count += 1
    folium.map.Marker(
    [lat, lng],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 11pt">%s</div>' % str(value['도로명']),
        )
    ).add_to(m)
print('7월 17일 강남구, 서초구 재비산먼지 청소도로위치 시각화')
m

lat 37.4842662461058 lng 127.003834177799
lat 37.4866606902992 lng 126.994293607948
7월 17일 강남구, 서초구 재비산먼지 청소도로위치 시각화
